# Image inference conversion notebook

## Notebook setup

In [ ]:
# noqa
import os
COLAB = 'DATALAB_DEBUG' in os.environ

if COLAB:
    !apt-get update
    !apt-get install git
    !git clone https://gist.github.com/oskopek/e27ca34cb2b813cae614520e8374e741 bstrap
    import bstrap.bootstrap as bootstrap
else:
    wd = %%pwd
    if wd.endswith('notebooks'):
        print('Current directory:', wd)
        %cd ..
        %pwd
    import resources.our_colab_utils.bootstrap as bootstrap

bootstrap.bootstrap(branch='master', packages='dotmap==1.2.20 keras==2.1.4 pydicom==1.0.2 Pillow==5.0.0')

if COLAB:
    !rm -rf bstrap

## Actual notebook

In [ ]:
# noqa
import csv
import os
from dotmap import DotMap

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pydicom
import skimage.transform
import tensorflow as tf

import resources.data.loader as loader
import resources.image_utils as imutils
import resources.synthetic_data as synth_data

%load_ext autoreload
%autoreload 2
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.cmap'] = 'gray' # 'viridis', 'gray'

## Define plotting

In [ ]:
import skimage.exposure


def show_img(img):
    img = img.copy()
    img = imutils.normalize(img, new_min=0, new_max=1)
    img = skimage.exposure.equalize_hist(img)
    img[0, 0] = 1.0
    #     img = skimage.exposure.adjust_log(img)
    #     img = imutils.normalize_gaussian(img, std=1/3)
    #     img = imutils.normalize(img, new_min=0, new_max=255)

    f = plt.figure(figsize=(16, 8))
    ax = f.add_subplot(1, 2, 1)
    ax2 = f.add_subplot(1, 2, 2)
    ax.imshow(img)
    ax2.hist(np.ravel(img))
    plt.show()


## Define helper functions

In [ ]:
def get_image(example, suffix='', name='image'):
    w = example.features.feature['width'].int64_list.value[0]
    h = example.features.feature['height'].int64_list.value[0]
    img_string = example.features.feature[name + suffix].bytes_list.value[0]
    img = np.frombuffer(img_string, dtype=np.float32)
    img = img.reshape(h, w)
    return img


def get_images(tfrecords_glob,
               options=tf.python_io.TFRecordOptions(compression_type=tf.python_io.TFRecordCompressionType.GZIP)):
    for file in tf.gfile.Glob(tfrecords_glob):
        for record in tf.python_io.tf_record_iterator(file, options=options):
            example = tf.train.Example()
            example.ParseFromString(record)
            yield get_image(example), get_image(
                    example, suffix='_gen'), get_image(
                            example, name='mask'), get_image(
                                    example, suffix='_gen', name='mask')


def to_png(matrix, path):
    img = matrix.copy()
    img = imutils.normalize_gaussian(img)
    img = imutils.normalize(img, new_min=0, new_max=255)
    img = Image.fromarray(img)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img.save(path)


In [ ]:
infdir = '/home/oskopek/tmp'
eval_dir = 'MaskTrue_BcdrInbreastFilterTrain_NoAugment_ICNR_nnUPSAMPLE_Lam0.0_88ksteps_model.ckpt-88417_inference_eval'
for cancer in ["cancer*", "healthy*"]:
    src_glob = os.path.join(infdir, eval_dir, "{}_gen.tfrecord".format(cancer))
    target_dir = os.path.join(infdir, eval_dir + "_png_{}".format(cancer[:-1]))
    print(src_glob, target_dir)
    tf.gfile.MakeDirs(target_dir)
    for i, (image, gen, mask, gen_mask) in enumerate(get_images(src_glob)):
        if i == 10:
            break
        show_img(image)
        show_img(gen)
#         to_png(image, os.path.join(target_dir, "{:05}_A_orig.png".format(i)))
#         to_png(mask, os.path.join(target_dir, "{:05}_B_mask.png".format(i)))
#         to_png(gen, os.path.join(target_dir, "{:05}_C_gen.png".format(i)))
#         to_png(gen_mask, os.path.join(target_dir, "{:05}_D_gen_mask.png".format(i)))

In [ ]:
img = image
img = img.copy()
img /= np.max(img)
img = skimage.exposure.adjust_gamma(img, gamma=2.0)

In [ ]:
img.max()